# Myeloid scRNAseq reference - Sensitivity Analysis Aug 2023


Aim: to understand if increasing the number of relevant genes will help finetyping myeloid cells 
Workflow:
  1. Load the full scRNAseq dataset
      - Delete any clusters that are indistinguishable with CosMx gene panel
  2. Fit the GLMM for every gene on all the cells
  3. Find relevant genes based on z-score
  4. Harmonize with genes selected from step 3
  5. Annotate cells, delete samples containing < 5 cells, remove cells with clashing labels (where the annotated labels dont match old labels)
  6. Re-harmonize QC'ed scRNAseq data to make sure everything looks good
  7. Integrate! Harmonize scRNAseq data with cosmx data (NOTE: use the same QC thresholds for every iteration)
  8. Let us see the results!



In [ ]:
options(warn = -1, verbose=FALSE)
#!/usr/bin/env Rscript 
library(dplyr)
library(Seurat)
library(httr)
library(readr)
library(pheatmap)
library(RColorBrewer)
library(ggplot2)
library(cowplot)
library(patchwork)
library(unixtools)
library(ggrepel)
library(repr)
library(ggmin)
library(harmony)
library(SeuratWrappers)
library(Nebulosa)
library(ggthemes)
library(purrr)
library(radiant.data)
library(presto)
library(pryr)
set_config(config(ssl_verifypeer = 0L))
ulimit::memory_limit(100000)
set.tempdir("/datastore/lucy/tmp/")
setwd("/datastore/lucy/CosMx")

# Load Korsunsky lab functions

In [ ]:
source("./R/utils.R")

In [ ]:
start_upR(clusterfiles = TRUE)

# Load synovial tissue single-cell data

In [ ]:
sc.syno<-readRDS("./cache/sc.syno.rds")

In [ ]:
dimlist(sc.syno)

In [ ]:
#Make sure cell ids in metadata match those in counts
all(rownames(sc.syno$metadata) == colnames(sc.syno$counts))

In [ ]:
head(sc.syno$metadata)

In [ ]:
rownames(sc.syno$metadata) <- sc.syno$metadata$cellID

In [ ]:
#Make sure cell ids in metadata match those in counts
all(rownames(sc.syno$metadata) == colnames(sc.syno$counts))

In [ ]:
head(sc.syno$metadata)

In [ ]:
sc.syno$metadata$sample <- sc.syno$metadata$Unique_ID

In [ ]:
sc.syno$metadata %>% 
    with(table(cosmx.myeloid))

In [ ]:
sc.syno$metadata %>% 
    with(table(cosmx.celltype))

In [ ]:
colnames(sc.syno$metadata)

In [ ]:
ncells_raw<-sc.syno$metadata %>% 
    subset(cosmx.celltype %in% c("STM","cDC")) %>% 
    with(table(cosmx.myeloid)) %>% 
    as.data.frame
ncells_raw

In [ ]:
head(rownames(sc.syno$metadata))

In [ ]:
rownames(sc.syno$metadata) <- sc.syno$metadata$cellID

# 2. Fit the global model

In [ ]:
presto.presto <- function (formula, design, response, size_varname, features = NULL, 
    effects_cov = c(""), ncore = 1, nsim = 100, family = "poisson", 
    min_sigma = 0, verbose = 0L) 
{
    if (is.null(features)) {
        features <- rownames(response)
    }
    if (family %in% c("poisson", "binomial", "nb")) {
        message("CAUTION: if using GLMM, make sure your counts are integers!")
    }
    design$EXPOSURE <- design[[size_varname]]
    fstr <- gsub(size_varname, "EXPOSURE", as.character(formula))
    formula <- as.formula(sprintf("%s~%s", fstr[[2]], fstr[[3]]), 
        env = .GlobalEnv)
    if (verbose > 0) {
        message("Set up models")
    }
    model_base <- fit_model.presto(formula, design, response[features[[1]], 
        ], family)
    priornames_df <- as.data.frame(VarCorr(model_base))[, 1:3]
    if (isGLMM(model_base)) {
        priornames_df <- rbind(priornames_df, tibble(grp = "Residual", 
            var1 = NA, var2 = NA))
    }
    has_offset <- !all(map_lgl(model_base@resp$offset, identical, 
        0))
    betanames_df <- make_betanames_df(model_base, has_offset)
    features <- intersect(features, rownames(response))
    if (ncore == 1) {
        future::plan(sequential)
    }
    else if (ncore %in% c(0, Inf)) {
        ncore <- availableCores()
        future::plan(multisession)
    }
    else {
        .ncore <<- ncore
        future::plan(future::multisession(workers = .ncore))
        rm(.ncore)
    }
    if (verbose > 0) {
        message("Learn the models")
    }
    lres <- furrr::future_map(features, glmm_uni, formula, design, 
        response, effects_cov, family, nsim, has_offset, min_sigma)
    names(lres) <- features
    lres <- lres[which(purrr::map_lgl(as.integer(map_int(lres, 
        "status")), identical, 0L))]
    if (verbose > 0) {
        message("Aggregate the results")
    }
    res <- collapse_lres(lres)
    if (verbose > 0) {
        message("Cleap up names")
    }
    covmat_names <- tibble(grpvar_orig = rownames(res$covmat)) %>% 
        left_join(subset(betanames_df, term %in% c("(Intercept)", 
            "Fixed"))) %>% dplyr::mutate(newname = case_when(is.na(grpvar) ~ 
        grpvar_orig, TRUE ~ as.character(glue::glue("{grpvar}.{grp}.{term}")))) %>% 
        with(newname)
    dimnames(res$covmat) <- list(covmat_names, covmat_names, 
        colnames(res$beta))
    res$betanames_df <- betanames_df
    res$priornames_df <- priornames_df
    res$meta_data <- design
    if (has_offset) {
        res$design <- list(EXPOSURE = model_base@resp$offset, 
            t(model_base@pp$X), model_base@pp$Zt) %>% purrr::reduce(Matrix::rbind2)
    }
    else {
        res$design <- list(t(model_base@pp$X), model_base@pp$Zt) %>% 
            purrr::reduce(Matrix::rbind2)
    }
    row.names(res$design) <- res$betanames_df$grp
    res$response <- response[names(lres), ]
    if (verbose > 0) {
        message("Compute gene means")
    }
    res <- genemeans.presto(res, xpm = 1e+06)
    res$has_offset <- has_offset
    res$family <- family
    res$size_varname <- size_varname
    res$nsim <- nsim
    res$formula_str <- as.character(formula)
    return(res)
}



In [ ]:
future::plan(multicore, workers=3)

In [ ]:
pb<-presto::collapse_counts(
    sc.syno$counts, 
    sc.syno$metadata, 
    c("sample", "cosmx.myeloid"), 
    min_cells_per_group = 3
)

In [ ]:
dimlist(pb)

In [ ]:
#These are internal functions i.e. unexported so we must access with :::
collapse_vecs <- presto:::collapse_vecs
collapse_mats <- presto:::collapse_mats
collapse_lres <- presto:::collapse_lres

In [ ]:
system.time({
    suppressWarnings({
        presto_res<-presto.presto(
            y ~ 1 + (1|cosmx.myeloid) + (1|cosmx.myeloid:sample) + (1|sample) + offset(logUMI), 
            pb$meta_data, 
            pb$counts_mat,
            size_varname = "logUMI", 
            effects_cov = "cosmx.myeloid",
            ncore = 3, 
            min_sigma = .05,
            family = "poisson",
            nsim = 1000
        )    
    })
})

In [ ]:
saveRDS(presto_res, "presto_res.cosmx.myeloid.rds")

## Make contrasts

In [ ]:
cellids_myeloid<-sc.syno$metadata %>% 
    subset(cosmx.celltype %in% c("STM","cDC")) %>% 
    rownames()

In [ ]:
sc.myeloid<-list()
sc.myeloid$counts<-sc.syno$counts[, cellids_myeloid]
sc.myeloid$metadata<-sc.syno$metadata[cellids_myeloid, ]

In [ ]:
unique.myeloid<-unique(sc.myeloid$metadata$cosmx.myeloid)

In [ ]:
unique.myeloid

In [ ]:
contrasts_mat<-make_contrast.presto(
    presto_res, 
    'cosmx.myeloid', 
    levels_contrast = unique.myeloid
)

In [ ]:

effects_marginal<-contrasts.presto(
    presto_res, 
    contrasts_mat, 
    one_tailed = FALSE
) %>% 
    dplyr::mutate(cluster = contrast) %>% 
    dplyr::mutate(
        ## convert stats to log2 for interpretability 
        logFC = sign(beta) * log2(exp(abs(beta))),
        SD = log2(exp(sigma)),
        zscore = logFC / SD
    ) %>%
    arrange(pvalue)

effects_marginal$fdr <- p.adjust(effects_marginal$pvalue, method = 'BH')


In [ ]:
effects_marginal

# 3. Find relevant genes based on z-scoring

In [ ]:
sc.syno_genes_max_zscores<-data.table(effects_marginal)[, .SD[order(-zscore)][1, ], by = feature][
    , .(feature, logFC, zscore)
]

In [ ]:
genes_use<-sc.syno_genes_max_zscores %>% top_n(wt = zscore, n = 50) %>% with(feature)

In [ ]:
length(genes_use)

# 4. Harmonize sc data with genes selected from step 3

## sample for one run

In [ ]:
all(colnames(sc.myeloid$counts) == rownames(sc.myeloid$metadata))

In [ ]:
table(sc.myeloid$metadata$cosmx.myeloid)

In [ ]:
dimlist(sc.myeloid)

In [ ]:
QC_gcmat(sc.myeloid$counts, 9, 12) %>% dim

In [ ]:
plot_genes_counts(sc.myeloid$counts, genes_use, gene_int = 9, count_int = 12)

In [ ]:
rm(sc.syno)
gc()

In [ ]:
rm(presto_res)

In [ ]:
gc()

## iterating over different number of relevant genes

In [ ]:
genes_use_list<-map(
    c(50, seq(100, nrow(sc.myeloid$counts), 100), nrow(sc.myeloid$counts)), 
    ~ data.frame(
        genes = sc.syno_genes_max_zscores %>% 
            top_n(wt = zscore, n = .x) %>% 
            with(unique(feature))
        ) %>% 
        mutate(source = paste0('top', .x))
    )

In [ ]:
map(genes_use_list, ~ length(.x$genes))

In [ ]:
batch<-c('sample','Experiment')
theta_harmony<-c(0,2) 
sigma_harmony<-0.2

sc.myeloid_meta<-map(genes_use_list, function(genes_use){
    
    sc.myeloid<-list()
    sc.myeloid$metadata<-sc.syno$metadata %>%  subset(cosmx.celltype %in% c("STM","cDC")) 
    sc.myeloid$counts<-sc.syno$counts[genes_use$genes, rownames(sc.myeloid$metadata)]
    
    print(dimlist(sc.myeloid))
    system.time({
        sc.myeloid.harmony<-QC_harmony_pipeline_normval(
            sc.myeloid, 
            ngenes_threshold = 9, 
            ncounts_threshold = 12, 
            do_cluster_after = TRUE,
            resolution_clustering = c(1.5, 2.5, 3.5),
            do_umap_after = TRUE,
            vars_use = batch,
            theta = theta_harmony,
            sigma = sigma_harmony,
            max.iter.harmony = 15,
            max.iter.cluster = 300,
            return_object = TRUE
        )

    sc.myeloid.harmony$sigma_harmony<-sigma_harmony
    sc.myeloid.harmony$vars_use<-batch
    sc.myeloid.harmony$theta_harmony<-theta_harmony    
    sc.myeloid.harmony$source = genes_use$source
    })
    
    return(sc.myeloid.harmony)
})
saveRDS(sc.myeloid_meta, "./cache/sc.myeloid_sensitivity.RDS")

## load

In [ ]:
sc.myeloid_meta<-readRDS("./cache/sc.myeloid_sensitivity.RDS")

In [ ]:
length(sc.myeloid_meta)

In [ ]:
(sc.myeloid_meta[[1]]$source[1])

In [ ]:
sc.myeloid_meta[[4]]$metadata %>% with(unique(cosmx.myeloid))

## Figures

In [ ]:
map(sc.myeloid_meta, function(sc.myeloid.harmony){
    pca_before<-plot_dim_red(
        dim_red_embeddings = sc.myeloid.harmony$pca_res$embeddings,
        clusters = rep(1, nrow(sc.myeloid.harmony$pca_res$embeddings)),
        metadata = sc.myeloid.harmony$metadata, 
        cell_id_colname = "cellID",
        color_by = "sample",
        plot_title = paste0("PCA before Harmony colored by sample - ", sc.myeloid.harmony$source[1], " genes"),
        dim_red_type = "PCA",
        size_points = 0.2
    )
    pca_after<-plot_dim_red(
        dim_red_embeddings = sc.myeloid.harmony$H,
        clusters = rep(1, nrow(sc.myeloid.harmony$pca_res$embeddings)),
        metadata = sc.myeloid.harmony$metadata, 
        cell_id_colname = "cellID",
        color_by = "sample",
        plot_title = paste0("PCA after Harmony colored by sample - ", sc.myeloid.harmony$source[1], " genes"),
        dim_red_type = "PCA",
        size_points = 0.2
    )
    
    fig.size(5, 10)
    pca_before + theme(legend.position = "none") |
    pca_after + theme(legend.position = "none")
})

In [ ]:
map(sc.myeloid_meta, function(sc.myeloid.harmony){
    pca_before<-plot_dim_red(
        dim_red_embeddings = sc.myeloid.harmony$pca_res$embeddings,
        clusters = rep(1, nrow(sc.myeloid.harmony$pca_res$embeddings)),
        metadata = sc.myeloid.harmony$metadata, 
        cell_id_colname = "cellID",
        color_by = "cosmx.myeloid",
        plot_title = paste0("PCA before Harmony colored by sample - ", sc.myeloid.harmony$source[1], " genes"),
        dim_red_type = "PCA",
        size_points = 0.2
    )
    pca_after<-plot_dim_red(
        dim_red_embeddings = sc.myeloid.harmony$H,
        clusters = rep(1, nrow(sc.myeloid.harmony$pca_res$embeddings)),
        metadata = sc.myeloid.harmony$metadata, 
        cell_id_colname = "cellID",
        color_by = "cosmx.myeloid",
        plot_title = paste0("PCA after Harmony colored by sample - ", sc.myeloid.harmony$source[1], " genes"),
        dim_red_type = "PCA",
        size_points = 0.2
    )
    
    fig.size(5, 20)
    pca_before | 
    pca_after 
})

In [ ]:
plot_dim_red <- function (dim_red_embeddings, clusters = NULL, metadata, cell_id_colname = "cellID", 
    color_by, plot_title = "title", dim_red_type, legend_posn = "right", 
    shape_points = ".", size_points = 0.1, point_type = "size", 
    legend_title_input = NULL, plot_labels = FALSE, shorten_labels = FALSE) 
{
    if (is.null(clusters)) 
        clusters = rep(1, nrow(dim_red_embeddings))
    plt_df <- dim_red_embeddings[, 1:2] %>% as.data.frame %>%
        purrr::set_names("V1", "V2") %>% 
        cbind(clusters) %>% cbind(metadata) %>% 
        rename(color_col = as.symbol(color_by)) %>% group_by(color_col) %>% 
        mutate(x_mid = median(V1), y_mid = median(V2), label_col = color_col) %>% 
        ungroup() %>% sample_frac(1)
    cols_plot <- generate_colors_tableau(plt_df$color_col)
    if (is.null(legend_title_input)) 
        legend_title_input = "color_col"
    if (plot_labels & shorten_labels) {
        plt_df <- plt_df %>% mutate(label_col = gsub("(.*?):.*", 
            "\\1", color_col))
    }
#     print(head(plt_df))
    p <- ggplot(data = plt_df, aes(V1, V2, color = color_col)) + 
        {
            if (point_type == "size") 
                geom_point(size = size_points)
        } + {
        if (point_type == "shape") 
            geom_point(shape = shape_points)
    } + ggtitle(plot_title) + scale_color_manual(values = cols_plot) + 
        {
            if (plot_labels) 
                geom_label_repel(data = plt_df %>% dplyr::select(starts_with(c("col", 
                  "x_mid", "y_mid", "label"))) %>% distinct, 
                  aes(x = x_mid, y = y_mid, label = label_col, 
                    fill = color_col), color = "black", size = 5, 
                  min.segment.length = 0, max.overlaps = Inf)
        } + {
        if (plot_labels) 
            scale_fill_manual(values = cols_plot)
    } + {
        if (plot_labels) 
            guides(fill = guide_legend(title = legend_title_input, 
                override.aes = list(color = NA)))
    } + {
        if (!plot_labels) 
            guides(color = guide_legend(title = legend_title_input, 
                override.aes = list(shape = 16, alpha = 1, size = 5)))
    } + xlab(paste0(dim_red_type, "_1")) + ylab(paste0(dim_red_type, 
        "_2")) + theme(plot.title = element_textbox_simple(margin = margin(10, 
        0, 10, 0)), legend.position = legend_posn, legend.text = element_textbox_simple(width = unit(5, 
        "cm")))
}

## UMAP

In [ ]:
map(sc.myeloid_meta, function(sc.myeloid.harmony){
    
    umap_before<-plot_dim_red(
        dim_red_embeddings = sc.myeloid.harmony$umap$embedding,
        clusters = rep(1, nrow(sc.myeloid.harmony$pca_res$embeddings)),
        metadata = sc.myeloid.harmony$metadata, 
        cell_id_colname = "cellID",
        color_by = "sample",
        plot_title = paste0("UMAP before Harmony colored by sample - ", sc.myeloid.harmony$source[1], " genes"),
        dim_red_type = "UMAP",
        size_points = 0.2
    )
    umap_after<-plot_dim_red(
        dim_red_embeddings = sc.myeloid.harmony$Humap$embedding,
        clusters = rep(1, nrow(sc.myeloid.harmony$pca_res$embeddings)),
        metadata = sc.myeloid.harmony$metadata, 
        cell_id_colname = "cellID",
        color_by = "sample",
        plot_title = paste0("UMAP after Harmony colored by sample - ", sc.myeloid.harmony$source[1], " genes"),
        dim_red_type = "UMAP",
        size_points = 0.2
    )
    
    fig.size(5, 10)
    umap_before + theme(legend.position = "none") |
    umap_after + theme(legend.position = "none")
    
})

### by cell type

In [ ]:
library(ggh4x)

In [ ]:
map(sc.myeloid_meta, function(sc.myeloid.harmony){
    
     umap_before<-plot_dim_red(
         dim_red_embeddings = sc.myeloid.harmony$umap$embedding,
         metadata = sc.myeloid.harmony$metadata, 
         cell_id_colname = "cellID",
         color_by = "cosmx.myeloid",
         plot_title = paste0("UMAP before Harmony colored by celltype - ", sc.myeloid.harmony$source[1], " genes"),
         dim_red_type = "UMAP",
         size_points = 0.1
     )
    umap_after<-plot_dim_red(
        dim_red_embeddings = sc.myeloid.harmony$Humap$embedding,
        clusters = rep(1, nrow(sc.myeloid.harmony$pca_res$embeddings)),
        metadata = sc.myeloid.harmony$metadata, 
        cell_id_colname = "cellID",
        color_by = "cosmx.myeloid",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid.harmony$source[1], " genes"),
        dim_red_type = "UMAP",
        size_points = 0.1
    )
   
    cells_cols<-generate_colors_tableau(sc.myeloid.harmony$metadata$new_cluster)
    strip<-strip_themed(background_x = elem_list_rect(fill = cells_cols))
    fig.size(5, 15)
    # umap_before + theme(legend.position = "none") |
    umap_after +
#         theme(
#             legend.text = element_textbox_simple(width = unit(5, "cm")), 
#             legend.key.height=unit(2, "cm"),
#             plot.title = element_textbox_simple(margin = margin(10, 0, 10, 0))
#         )  |
    umap_after + 
        facet_wrap2(~ color_col, strip = strip) + 
        theme(
            legend.position = "none",
            strip.text = element_textbox_simple(size = 8, width = unit(4, "cm"))
             ) +
        NULL
    
})

# Integrate with CosMx data

## load CosMx data

In [ ]:
cosmx.syno<-readRDS("./cache/CosMxcoarseGrainharmonyObj_markers.RDS")

In [ ]:
dimlist(cosmx.syno) 

In [ ]:
head(cosmx.syno$metadata)

In [ ]:
celltype<-c("Myeloid","DC")

In [ ]:
head(cosmx.syno$metadata)

In [ ]:
cosmx.syno$metadata<-cosmx.syno$metadata %>% 
    dplyr::mutate(
        celltype.custom = celltype.coarse,
        SampleFOV = paste0(SampleID, '_', FOV)
    )
    

## filter

In [ ]:
cell.ids<-cosmx.syno$metadata %>% 
    filter(celltype.custom %in% celltype) %>% 
    dplyr::select(cellID)

In [ ]:
cosmx.syno$metadata %>% filter(cellID %in% cell.ids$cellID) %>% head

In [ ]:
cosmx.myeloid<-list()
cosmx.myeloid$counts<-cosmx.syno$counts[, colnames(cosmx.syno$counts) %in% cell.ids$cellID]
cosmx.myeloid$metadata<-cosmx.syno$metadata %>% subset(cellID %in% cell.ids$cellID)

In [ ]:
dimlist(cosmx.myeloid)

In [ ]:
all(cosmx.myeloid$metadata$cellID == colnames(cosmx.myeloid$counts))

In [ ]:
fig.size(5, 5)
map(sc.myeloid_meta, function(sc.myeloid.harmony){
    genes_use<-rownames(sc.myeloid.harmony$counts)
    cosmxdata<-list()
    cosmxdata$counts<-cosmx.myeloid$counts[genes_use, ]
    cosmxdata$metadata<-cosmx.myeloid$metadata %>% subset(cellID %in% colnames(cosmxdata$counts))
    
    data_qc<-QC_gcmat(cosmxdata$counts, gene_thresh = 3, count_thresh = 7)
    
    cell_count<-data.frame(frac_cells = ncol(data_qc)/ncol(cosmxdata$counts), ngenes = length(genes_use))
    
    }) %>% 
    rbindlist %>% 
    ggplot(aes(ngenes, frac_cells)) + 
        geom_point() + 
        ggtitle("Fraction of cells after QC vs number of relevant genes") + 
        theme(plot.title = element_textbox_simple()) + 
        ylab("Fraction of cells remaining after QC")
    
#     fig.size(12, 20)
# tibble(
#     gene = genes_use, 
#     cosmx = prop.table(rowSums(cosmxdata$counts[genes_use, ])), 
#     amp = prop.table(rowSums(sc.syno$counts[genes_use, ]))
# ) %>% 
#     ggplot(aes(amp, cosmx)) + 
#         geom_point() + 
#         geom_text_repel(aes(label = gene), max.overlaps = Inf) + 
#         geom_abline() + 
#         geom_smooth() + 
#         scale_x_log10() + scale_y_log10() + 
        
#         NULL




In [ ]:
fig.size(5, 5)
map(sc.myeloid_meta, function(sc.myeloid.harmony){
    genes_use<-rownames(sc.myeloid.harmony$counts)
    cosmxdata<-list()
    cosmxdata$counts<-cosmx.myeloid$counts[genes_use, ]
    cosmxdata$metadata<-cosmx.myeloid$metadata %>% subset(cellID %in% colnames(cosmxdata$counts))
    
    data_qc<-QC_gcmat(cosmxdata$counts, gene_thresh = 10, count_thresh = 15)
    
    cell_count<-data.frame(frac_cells = ncol(data_qc)/ncol(cosmxdata$counts), ngenes = length(genes_use))
    
    }) %>% 
    rbindlist %>% 
    ggplot(aes(ngenes, frac_cells)) + 
        geom_point() + 
        ggtitle("Fraction of cells after QC vs number of relevant genes") + 
        theme(plot.title = element_textbox_simple()) + 
        ylab("Fraction of cells remaining after QC")

# I will integrate with 400 genes

## Clean up and annotate sc.syno with 400 genes

In [ ]:
sc.myeloid_meta[[4]]$metadata %>% with(table(cosmx.myeloid))

In [ ]:
dimlist(sc.myeloid_meta[[5]])

In [ ]:
sc.myeloid<-sc.myeloid_meta[[5]]

In [ ]:
sc.myeloid$metadata %>% cbind(sc.myeloid$Humap$clusters) %>% with(table(Clust1.5))

In [ ]:
head(sc.myeloid$Humap$clusters)

In [ ]:
pclust1<-plot_dim_red(
        dim_red_embeddings = sc.myeloid$Humap$embedding,
        clusters = sc.myeloid$Humap$clusters,
        metadata = sc.myeloid$metadata, 
        cell_id_colname = "cellID",
        color_by = "Clust1.5",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid$source[1], " genes"),
        dim_red_type = "UMAP",
        plot_labels = TRUE
    )
pclust2<-plot_dim_red(
        dim_red_embeddings = sc.myeloid$Humap$embedding,
        clusters = sc.myeloid$Humap$clusters,
        metadata = sc.myeloid$metadata, 
        cell_id_colname = "cellID",
        color_by = "Clust2.5",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid$source[1], " genes"),
        dim_red_type = "UMAP",
        plot_labels = TRUE
    )
pclust3<-plot_dim_red(
        dim_red_embeddings = sc.myeloid$Humap$embedding,
        clusters = sc.myeloid$Humap$clusters,
        metadata = sc.myeloid$metadata, 
        cell_id_colname = "cellID",
        color_by = "Clust3.5",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid$source[1], " genes"),
        dim_red_type = "UMAP",
        plot_labels = TRUE
    )
ptype<-plot_dim_red(
        dim_red_embeddings = sc.myeloid$Humap$embedding,
        clusters = sc.myeloid$Humap$clusters,
        metadata = sc.myeloid$metadata, 
        cell_id_colname = "cellID",
        color_by = "cosmx.myeloid",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid$source[1], " genes"),
        dim_red_type = "UMAP"
    )

In [ ]:
fig.size(5, 9)
ptype 
fig.size(5, 9)
pclust1
fig.size(5, 9)
pclust2
fig.size(5, 9)
pclust3

In [ ]:
fig.size(7, 15)
pclust1 + facet_wrap(~ color_col)

In [ ]:
fig.size(7, 10)
sc.myeloid$metadata %>% 
    cbind(sc.myeloid$Humap$clusters) %>% 
    with(table(cosmx.myeloid, Clust1.5)) %>% 
    prop.table(1) %>% 
    prop.table(2) %>% 
    Heatmap

Higher resolution (3.5) still does not even split DC1 from DC2 so we need to isolate and run seperately so just return to 1.5 resolution to make things easier

In [ ]:
head(sc.myeloid$metadata)

In [ ]:
sc.myeloid$metadata$Clust1.5<-sc.myeloid$Humap$clusters$`Clust1.5`

In [ ]:
head(sc.myeloid$metadata)

## Subset DCs and recluster 

In [ ]:
SPP1subclustering <- subcluster_cells(sc.myeloid$counts, sc.myeloid$metadata, sc.myeloid$Humap$snn, 
                             "Clust1.5", 4, c(0.2,0.4))

In [ ]:
head(SPP1subclustering)

In [ ]:
merged.clusters <- merge_clusters(sc.myeloid$metadata, SPP1subclustering, "Clust1.5", "finetype.Clust0.4")

In [ ]:
sc.myeloid$metadata <- merged.clusters

In [ ]:
head(merged.clusters)

In [ ]:
DCsubclustering <- subcluster_cells(sc.myeloid$counts, sc.myeloid$metadata, sc.myeloid$Humap$snn, 
                             "Clust1.5", 8, c(0.2,0.3,0.4,0.5))

In [ ]:
head(DCsubclustering)

In [ ]:
merged.clusters <- merge_clusters(sc.myeloid$metadata, DCsubclustering, "Clust1.5", "finetype.Clust0.5")

In [ ]:
sc.myeloid$metadata <- merged.clusters

In [ ]:
head(merged.clusters)

In [ ]:
unique(merged.clusters$Clust1.5)

In [ ]:
plot1 <- plot_dim_red(
        dim_red_embeddings = sc.myeloid$Humap$embedding,
        metadata = sc.myeloid$metadata, 
        cell_id_colname = "cellID",
        color_by = "Clust1.5",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid$source[1], " genes"),
        dim_red_type = "UMAP"
    )

In [ ]:
fig.size(7,10)

plot1

In [ ]:
fig.size(7, 15)
plot1 + facet_wrap(~ color_col)

In [ ]:
fig.size(7, 15)
sc.myeloid$metadata %>% 
    cbind(sc.myeloid$Humap$clusters) %>% 
    with(table(cosmx.myeloid, Clust1.5)) %>% 
    prop.table(1) %>% 
    prop.table(2) %>% 
    Heatmap

In [ ]:
monoDCsubclustering <- subcluster_cells(sc.myeloid$counts, sc.myeloid$metadata, sc.myeloid$Humap$snn, 
                             "Clust1.5", 1, c(0.3))

In [ ]:
head(monoDCsubclustering)

In [ ]:
merged.clusters <- merge_clusters(sc.myeloid$metadata, monoDCsubclustering, "Clust1.5", "Clust0.3")

In [ ]:
sc.myeloid$metadata <- merged.clusters

In [ ]:
head(merged.clusters)

In [ ]:
unique(merged.clusters$Clust1.5)

In [ ]:
plot1 <- plot_dim_red(
        dim_red_embeddings = sc.myeloid$Humap$embedding,
        metadata = sc.myeloid$metadata, 
        cell_id_colname = "cellID",
        color_by = "Clust1.5",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid$source[1], " genes"),
        dim_red_type = "UMAP"
    )

In [ ]:
fig.size(7,10)

plot1

In [ ]:
fig.size(7, 15)
plot1 + facet_wrap(~ color_col)

In [ ]:
fig.size(7, 15)
sc.myeloid$metadata %>% 
    cbind(sc.myeloid$Humap$clusters) %>% 
    with(table(cosmx.myeloid, Clust1.5)) %>% 
    prop.table(1) %>% 
    prop.table(2) %>% 
    Heatmap

## Annotations

In [ ]:
annotations<-sc.myeloid$metadata %>% 
    cbind(sc.myeloid$Humap$clusters) %>% 
    with(table(cosmx.myeloid, Clust1.5)) %>% 
    prop.table(1) %>% 
    prop.table(2) %>% 
    as.data.frame %>% 
    group_by(Clust1.5) %>% 
    mutate(maxFreq = max(Freq)) %>% 
    subset(Freq == maxFreq) %>% 
    mutate(
        cosmx.myeloid = as.character(cosmx.myeloid),
        Clust1.5 = as.character(Clust1.5)
    ) %>% 
    dplyr::select(cosmx.myeloid, Clust1.5) %>% 
    rename(fine_type = cosmx.myeloid)

In [ ]:
annotations

In [ ]:
ann_auto<-annotations %>% 
    group_by(fine_type) %>% 
    summarise(clusters = paste(Clust1.5, collapse = ".")) %>%  
    mutate(fine_type = gsub("(.*?):.*", "\\1", fine_type))

In [ ]:
ann_auto

In [ ]:
sc.myeloid$metadata<-sc.myeloid$metadata %>% 
    left_join(annotations, by = "Clust1.5")

In [ ]:
fig.size(7, 7)
sc.myeloid$metadata %>% 
    with(table(cosmx.myeloid, fine_type)) %>% 
    prop.table(1) %>% 
    prop.table(2) %>% 
    Heatmap

In [ ]:
table(sc.myeloid$metadata$fine_type)

In [ ]:
plot1 <- plot_dim_red(
        dim_red_embeddings = sc.myeloid$Humap$embedding,
        metadata = sc.myeloid$metadata, 
        cell_id_colname = "cellID",
        color_by = "fine_type",
        plot_title = paste0("UMAP after Harmony colored by celltype - ", sc.myeloid$source[1], " genes"),
        dim_red_type = "UMAP"
    )

In [ ]:
sc.myeloid$metadata

In [ ]:
fig.size(7,10)

plot1

## delete clashing cells

In [ ]:
sc.myeloid$metadata<-sc.myeloid$metadata %>% 
    subset(cosmx.myeloid == fine_type)
sc.myeloid$counts<-sc.myeloid$counts[, colnames(sc.myeloid$counts) %in% sc.myeloid$metadata$cellID]

In [ ]:
dimlist(sc.myeloid)

In [ ]:
all(colnames(sc.myeloid$counts) == sc.myeloid$metadata$cellID)

In [ ]:
table(sc.myeloid$metadata$fine_type) %>% as.data.frame 

## UMAP before and after merging labels

In [ ]:
sc_before_merge<-sc.myeloid_meta[[5]]

In [ ]:
umap_before_clean<-plot_dim_red(
        dim_red_embeddings = sc_before_merge$Humap$embedding,
        clusters = rep(1, nrow(sc_before_merge$pca_res$embeddings)),
        metadata = sc_before_merge$metadata, 
        cell_id_colname = "cellID",
        color_by = "cosmx.myeloid",
        plot_title = "UMAP of reference cells with relevant genes colored by their annotations",
        dim_red_type = "UMAP",
        size_points = 0.1
    )


In [ ]:
cols_pal<-tableau_color_pal("Tableau 20")(19)

In [ ]:
names(cols_pal)<-unique(sc_before_merge$metadata$cosmx.myeloid) %>% sort

In [ ]:
cols_pal

In [ ]:
fig.size(6, 16)
umap_before_clean + 
    labs(color = "celltype")  |

ptype + 
    labs(color = "celltype") + 
    ggtitle("UMAP of reference cells with relevant genes after cleanup") +
    scale_color_manual(values = cols_pal) 


In [ ]:
ncells_type<-sc.myeloid$metadata %>% with(table(fine_type)) %>%  as.data.frame 
ncells_type

# downsample ref

In [ ]:
sc.myeloid_down<-list()
sc.myeloid_down$metadata<-sc.myeloid$metadata %>% 
    dplyr::group_by(fine_type) %>% 
    dplyr::sample_n(min(n(), round(median(ncells_type$Freq))))
sc.myeloid_down$counts<-sc.myeloid$counts[, sc.myeloid_down$metadata$cellID]

In [ ]:
dimlist(sc.myeloid)

In [ ]:
dimlist(sc.myeloid_down)

### downsampling bring us down to 21223 cells from 9647 

In [ ]:
batch<-c("sample")
cluster_res<-c(1.5, 2.5)
theta_harmony<-c(0) 
sigma_harmony<-0.2
system.time({
    sc.myeloid.harmony_down<-QC_harmony_pipeline_normval(
        sc.myeloid_down, 
        ngenes_threshold = 9, 
        ncounts_threshold = 12, 
        do_cluster_after = TRUE,
        do_umap_after = TRUE,
        resolution_clustering = cluster_res, 
        clustering_ncores = 5,
        vars_use = batch,
        theta = theta_harmony,
        sigma = sigma_harmony,
        max.iter.harmony = 7,
        max.iter.cluster = 50,
        return_object = TRUE
    )

    sc.myeloid.harmony_down$sigma_harmony<-sigma_harmony
    sc.myeloid.harmony_down$vars_use<-batch
    sc.myeloid.harmony_down$theta_harmony<-theta_harmony    

    })


# cache

## add the df with original cell fractions to downsampled oect

In [ ]:
sc.syno<-readRDS("./cache/sc.syno.rds")

In [ ]:
ncells_raw<-sc.syno$metadata %>% 
    subset(cosmx.celltype %in% c("STM","cDC")) %>% 
    with(table(cosmx.myeloid)) %>% 
    as.data.frame
ncells_raw

In [ ]:
sc.myeloid.harmony_down$ncells_raw<-ncells_raw

In [ ]:
sc.myeloid.harmony_down$metadata$cosmx400.myeloid <- sc.myeloid.harmony_down$metadata$fine_type

In [ ]:
saveRDS(sc.myeloid.harmony_down, "./cache/sc.myeloid.harmony_down.RDS")

# Viz before QC, after QC, and after downsampling

In [ ]:
p_downsampled_newref<-plot_dim_red(
    sc.myeloid.harmony_down$Humap$embedding,
    metadata = sc.myeloid.harmony_down$metadata, 
    color_by = "fine_type", 
    plot_title = "UMAP new annotations", 
    dim_red_type = "UMAP"
    )

p_downsampled<-plot_dim_red(
    sc.myeloid.harmony_down$Humap$embedding,
    metadata = sc.myeloid.harmony_down$metadata, 
    color_by = "cosmx.myeloid", 
    plot_title = "UMAP ground truth annotations", 
    dim_red_type = "UMAP"
    )

In [ ]:
fig.size(5, 15)
p_downsampled_newref | p_downsampled

In [ ]:
fig.size(5, 22)
p<-umap_before_clean + 
    labs(color = "celltype")  |

ptype + 
    labs(color = "celltype") +
    ggtitle("UMAP of reference cells with relevant genes after cleanup") +
    scale_color_manual(values = cols_pal) |

p_downsampled  + 
    ggtitle("UMAP of reference cells with relevant genes after downsampling") +
    scale_color_manual(values = cols_pal) 

In [ ]:
print(p, vp=grid::viewport(width = unit(22, 'inch'), height = unit(5, 'inch')))

In [ ]:
ls()

In [ ]:
rm(list = ls())